In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_7859/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)


    def keep_random_models(self, n_models=3):
        # Extract block part from 'model' column
        self.results_df['block'] = self.results_df['model'].apply(lambda x: '_'.join(x.split('_')[1:3]))
        
        # Group by 'block' and keep n_models randomly from each group
        def select_random_models(group):
            return group.sample(n=min(n_models, len(group)), random_state=1)
        
        # Apply the function and ensure grouping columns are excluded from the final DataFrame
        grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)
        self.results_df = grouped.reset_index(drop=True).drop(columns=['block'])

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.keep_random_models()
        self.results_df = self.results_df.sort_values(by="r2_sup", ascending=False, ignore_index=True)
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



/tmp/ipykernel_7859/3323087823.py:36: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = self.results_df.groupby('block', group_keys=False).apply(select_random_models)


,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_3_10_4,0.965654,0.731541,0.750103,0.873739,0.110259,1.795188,0.893866,0.369027,0.699969,0.332053,0.853459,0.337282,102.409847,208.954824,"Hidden Size=[12], regularizer=0.02, learning_r..."
1,model_3_10_6,0.963294,0.730587,0.746231,0.858962,0.117838,1.801565,0.907717,0.412215,0.708025,0.343275,0.843386,0.348681,102.276891,208.821868,"Hidden Size=[12], regularizer=0.02, learning_r..."
2,model_3_11_4,0.961043,0.729881,0.733609,0.883230,0.125062,1.806284,0.952865,0.441262,0.790331,0.353641,0.833785,0.359210,102.157896,208.702872,"Hidden Size=[12], regularizer=0.02, learning_r..."
3,model_2_47_7,0.956356,0.729597,0.779569,0.902177,0.140109,1.808188,0.788470,0.332719,1.484120,0.374311,0.909897,0.380206,69.930668,141.685448,"Hidden Size=[8], regularizer=0.02, learning_ra..."
4,model_2_48_2,0.957770,0.729544,0.777155,0.893977,0.135571,1.808538,0.797105,0.181750,1.434252,0.368200,0.912815,0.373999,69.996515,141.751295,"Hidden Size=[8], regularizer=0.02, learning_ra..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,model_2_3_5,0.858861,0.584019,0.632768,0.858759,0.453095,2.781668,1.313569,0.265085,1.384432,0.673123,0.708617,0.683724,67.583306,139.338086,"Hidden Size=[8], regularizer=0.02, learning_ra..."
344,model_2_3_4,0.856932,0.582904,0.631143,0.862228,0.459289,2.789121,1.319380,0.258575,1.355015,0.677709,0.704634,0.688382,67.556150,139.310930,"Hidden Size=[8], regularizer=0.02, learning_ra..."
345,model_2_4_5,0.869940,0.581241,0.653351,0.877020,0.417529,2.800246,1.239943,0.784148,1.318513,0.646165,0.731490,0.656341,67.746802,139.501582,"Hidden Size=[8], regularizer=0.02, learning_ra..."
346,model_1_1_0,0.845111,0.580573,0.708379,0.236613,0.497239,2.804711,1.043111,0.373512,2.009508,0.705152,0.789087,0.716257,35.397367,72.361951,"Hidden Size=[4], regularizer=0.02, learning_ra..."
